In [1]:
import os
import numpy as np
import pandas as pd
import math
from pyomo.environ import *
from collections import defaultdict
from pyomo.opt import SolverFactory
from datetime import timedelta

In [8]:
raw = pd.read_csv('./data/project 4 dataset/init_data.csv', header=0, index_col=0)
raw.loc[51, 'StateCode'] = 'SNS'
raw.loc[51, 'Available capacity'] = 12000
raw.loc[51, 'nb_idx'] = str([i for i in range(51)])
raw

,StateCode,State,nb,nb_idx,Available capacity
0,AK,Alaska,['WA'],[46],200
1,AL,Alabama,"['FL', 'GA', 'MS', 'TN']","[8, 9, 24, 41]",1344
2,AR,Arkansas,"['LA', 'MO', 'MS', 'OK', 'TN', 'TX']","[17, 23, 24, 35, 41, 42]",500
3,AZ,Arizona,"['CA', 'CO', 'NM', 'NV', 'UT']","[4, 5, 31, 32, 43]",1500
4,CA,California,"['AZ', 'HI', 'NV', 'OR']","[3, 10, 32, 36]",11036
5,CO,Colorado,"['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY']","[3, 15, 28, 31, 35, 43, 49]",600
6,CT,Connecticut,"['MA', 'NY', 'RI']","[18, 33, 38]",1000
7,DE,Delaware,"['MD', 'NJ', 'PA']","[19, 30, 37]",400
8,FL,Florida,"['AL', 'GA']","[1, 9]",4000
9,GA,Georgia,"['AL', 'FL', 'NC', 'SC', 'TN']","[1, 8, 26, 39, 41]",1006


In [11]:
first_date = '2020/03/25'
t = 7
T = 14
P = 10
lbd = 0.1

S = 52
states_name = raw['State']
SNS_stock = 12000

init_ratio = 0.6
flow_bound_ratio = 0.25
stock_bound_ratio = 0.5

In [12]:
#first iteration values
data = raw.rename(columns={'Available capacity': "stock_self"})
data['stock_self'] = init_ratio * data['stock_self']
data['stock_nb'] = None
for i in range(S):
    data.loc[i, 'stock_nb'] = str([0 for _ in eval(data.loc[i, 'nb'])])
data['stock_sns'] = [0 for _ in range(S)]
data

TypeError: eval() arg 1 must be a string, bytes or code object

In [5]:
idx = 0
flow_mapping = {}
flow_mapping_rev = {}
out_flow = defaultdict(list)
in_flow = defaultdict(list)

for i in range(S):
    nbs = eval(data.iloc[i, 3])
    out_flow[i] = nbs
    for n in nbs:
        flow_mapping[(i, n)] = idx
        flow_mapping_rev[idx] = (i, n)
        in_flow[n].append(i)
        idx += 1

in_flow[51] = []
out_flow[51] = list(range(51))
for n in range(S):
    flow_mapping[(51, n)] = idx
    flow_mapping_rev[idx] = (51, n)
    idx += 1

In [6]:
model = ConcreteModel()

#Sets
F = idx
model.time_set = list(range(1, t + 1))
model.state_set = list(range(S))
model.state_p_set = list(range(S + 1))
model.flow_set = list(range(F))
model.flow_no_sns_set = list(range(F - S))

In [7]:
# instantiate Param
ini_self = [round(i) for i in data['stock_self']]
ini_nb = defaultdict(list)
for i in range(S):
    ini_nb[i] = eval(data.loc[i, 'stock_nb'])
ini_SNS = [round(i) for i in data['stock_sns']]

# Parameters
model.demand = Param(model.state_set, model.time_set, within=NonNegativeReals, mutable=True)

model.U = [round(flow_bound_ratio * ini_self[flow_mapping_rev[j][0]]) for j in model.flow_no_sns_set] + [round(flow_bound_ratio * init_ratio * SNS_stock) for _ in range(S)]
model.G = [round(stock_bound_ratio * i) for i in ini_self]

In [8]:
#date = pd.to_datetime(first_date) + timedelta(21)
date = pd.to_datetime(first_date) 
for i in range(S):
    state = data.loc[i, 'State']
    df = pd.read_csv('../../IHME/{}/{}.csv'.format(date.strftime('%m%d'), state), header=0, index_col=0)
    for tt in range(t):
        model.demand[i, tt + 1].value = df.iloc[tt, 1]

In [9]:
# Var
model.s0 = Var(model.state_p_set, within=NonNegativeReals)
model.s0_nb = Var(model.flow_set, within=NonNegativeReals)
model.x = Var(model.flow_set, within=NonNegativeReals)
model.delta = Var(model.state_set, model.time_set, within=NonNegativeReals)

In [10]:
#Constraints
model.stock_self = Constraint(model.state_set, rule=lambda model, j: model.s0[j] + sum(model.x[flow_mapping[(j, i)]] for i in out_flow[j]) == ini_self[j])
model.stock_SNS_self = Constraint(rule=lambda model: model.s0[51] + sum(model.x[flow_mapping[(51, i)]] for i in out_flow[51]) == init_ratio * SNS_stock)

model.stock_nb = ConstraintList()
for j in model.state_set:
    for dum, i in enumerate(out_flow[j]):
        f = flow_mapping[(i, j)] 
        model.stock_nb.add(model.s0_nb[f] - model.x[f] == ini_nb[j][dum])
    f = flow_mapping[(51, j)] 
    model.stock_nb.add(model.s0_nb[f] - model.x[f] == ini_SNS[j])
        
model.delta_bound = ConstraintList()
for j in model.state_set:
    for tt in model.time_set:
        total_stock = model.s0[j] + model.s0_nb[flow_mapping[(51, j)] ] + sum(model.s0_nb[flow_mapping[(i, j)]] for i in out_flow[j])
        model.delta_bound.add(model.delta[j, tt] + total_stock >= model.demand[j, tt])
        

model.s0_bound = Constraint(model.state_set, \
                                  rule=lambda model, j: model.s0[j] >= model.G[j])
model.x_bound = Constraint(model.flow_set, \
                                  rule=lambda model, j: model.x[j] <= model.U[j])

In [11]:
#Objective
model.unmet = Expression(initialize=(sum(model.delta[s, tt] for s in model.state_set for tt in model.time_set)))
model.penalty = Expression(initialize=(sum(lbd*model.s0_nb[i] for i in model.flow_no_sns_set)))
model.obj = Objective(expr = model.unmet + model.penalty, sense = minimize)

In [12]:
SolverFactory('cplex_direct').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: 
  Lower bound: 7.025068740000002
  Upper bound: 7.025068740000002
  Number of objectives: 1
  Number of constraints: 1006
  Number of variables: 955
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 955
  Number of nonzeros: None
  Sense: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: CPLEX 12.8.0.0
  Status: ok
  Wallclock time: 0.002009868621826172
  Termination condition: optimal
# ----------------------------------------------------------
#   Solution Information
# ------------

In [13]:
data['stock_all'] = [0 for _ in range(S)] 
for i in model.state_set:
    data.loc[i, 'stock_self'] = round(model.s0[i].value)
    data.loc[i, 'stock_all'] += data.loc[i, 'stock_self']
    
for i in model.state_set:
    l = eval(data.loc[i, 'nb_idx'])
    new_nb = []
    for n in l:
        j = flow_mapping[(n, i)]
        new_nb.append(round(model.s0_nb[j].value))
    data.loc[i, 'stock_nb'] = str(new_nb)
    data.loc[i, 'stock_all'] += sum(new_nb)

for i in model.state_set:
    j = flow_mapping[(51, i)]
    data.loc[i, 'stock_sns'] = round(model.s0_nb[j].value)
    data.loc[i, 'stock_all'] += data.loc[i, 'stock_sns']

In [14]:
data

,StateCode,State,nb,nb_idx,stock_self,stock_nb,stock_sns,stock_all
0,AK,Alaska,['WA'],[46],120.0,[0],0,120.0
1,AL,Alabama,"['FL', 'GA', 'MS', 'TN']","[8, 9, 24, 41]",806.0,"[0, 0, 0, 0]",0,806.0
2,AR,Arkansas,"['LA', 'MO', 'MS', 'OK', 'TN', 'TX']","[17, 23, 24, 35, 41, 42]",300.0,"[0, 0, 0, 0, 0, 0]",0,300.0
3,AZ,Arizona,"['CA', 'CO', 'NM', 'NV', 'UT']","[4, 5, 31, 32, 43]",900.0,"[0, 0, 0, 0, 0]",0,900.0
4,CA,California,"['AZ', 'HI', 'NV', 'OR']","[3, 10, 32, 36]",6622.0,"[0, 0, 0, 0]",0,6622.0
5,CO,Colorado,"['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY']","[3, 15, 28, 31, 35, 43, 49]",360.0,"[0, 0, 0, 0, 0, 0, 0]",0,360.0
6,CT,Connecticut,"['MA', 'NY', 'RI']","[18, 33, 38]",450.0,"[0, 0, 0]",0,450.0
7,DE,Delaware,"['MD', 'NJ', 'PA']","[19, 30, 37]",240.0,"[0, 0, 0]",0,240.0
8,FL,Florida,"['AL', 'GA']","[1, 9]",2400.0,"[0, 0]",0,2400.0
9,GA,Georgia,"['AL', 'FL', 'NC', 'SC', 'TN']","[1, 8, 26, 39, 41]",604.0,"[0, 0, 0, 0, 0]",0,604.0


In [16]:
model.obj()

7.025068740000002

In [411]:
data.to_csv(date.strftime('%m%d') + '.csv')

In [10]:
prediction[states_name[j]].iloc[t - 1, 1]

8.106127518402928

# With SNS

In [8]:
#open the cor file
prediction = {}
for d in range(1):
    date = ('_').join(dates[d].split('/'))
    T = (pd.to_datetime(dates[d + 1]) - pd.to_datetime(dates[d])).days
    for i in range(S):
        prediction[states_name[i]] = pd.read_csv('../IHME_data/' + date + '/' + states_name[i] + '.csv', header=0, index_col=0)
    
    
    base_dir = os.getcwd()
    prob_name = 'det_exp_{}'.format(date)

    f_path = os.path.join(base_dir, '../input/deterministic/' + prob_name + '.lp')
    f = open(f_path, 'w')

    f.write('Minimize \n')
    f.write(' obj: ')
    for j in range(S):
        for t in range(1, T + 1):
            if t or j: 
                f.write(' +')
            f.write(' delta_{}_{}'.format(j, t))
    f.write(' \n')

    f.write('subject To \n')
    for j in range(S):
        nbs = eval(data.loc[j, 'nb_idx'])
        f.write(' s_{}_0 - '.format(j))
        f.write(' - '.join(['x_{}_{}'.format(i, j) for i in nbs]))
        f.write(' - x_51_{}'.format(j))
        f.write(' + ')
        f.write(' + '.join(['x_{}_{}'.format(j, i) for i in nbs]))
        f.write(' = {}'.format(int(a[j])))
        f.write(' \n')
        
    f.write(' s_51_0 +')
    f.write(' + '.join(['x_51_{}'.format(i) for i in range(S)]))
    f.write(' = {}'.format(a[51]))
    f.write('\n')

    for j in range(S):
        for t in range(1, T + 1):
            dmd = math.ceil(prediction[states_name[j]].iloc[t - 1, 1])
            #print(date + states_name[j] + str(t))
            f.write(' delta_{}_{} + s_{}_0>= {}\n'.format(j, t, j, dmd))


    f.write('Bounds \n')
    for j in range(S):
        G = math.ceil(min(a[j], max(prediction[states_name[j]]['InvVen_mean'])))
        f.write(' {} <= s_{}_0 \n'.format(G, j))
    f.write(' 0 <= s_51_0 \n')

    for i in range(S + 1):
        nbs = eval(data.loc[i, 'nb_idx'])
        for j in nbs:
            f.write(' 0 <= x_{}_{} <= {}\n'.format(i, j, math.ceil(0.2 * a[i])))

    for j in range(S):
        for t in range(1, T + 1):
            f.write(' 0 <= delta_{}_{}\n'.format(j, t))

    f.write('End\n')